In [42]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Datei einlesen
eco = gpd.read_file("../../data/Rohdaten/messstandorte_verkehrszaehlungen_geojson/data/taz.view_eco_standorte.json")

# Grundstruktur prüfen
print("Datentypen:")
print(eco.dtypes)

print("\nNullwerte pro Spalte:")
print(eco.isnull().sum().sort_values(ascending=False))

print("\nForm:", eco.shape)

print("\nErste 5 Zeilen:")
print(eco.head())

print("\nEindeutige Werte pro Spalte:")
print(eco.nunique().sort_values(ascending=False))

print("\nDoppelte Zeilen:", eco.duplicated().sum())
print("Doppelte Geometrien:", eco.geometry.duplicated().sum())

print("\nSpalten mit nur einem Wert:")
print(eco.columns[eco.nunique() <= 1])


Datentypen:
abkuerzung           object
bezeichnung          object
bis                  object
fk_zaehler           object
id1                   int32
richtung_in          object
richtung_out         object
von                  object
objectid             object
korrekturfaktor     float64
geometry           geometry
dtype: object

Nullwerte pro Spalte:
bis                26
richtung_out        4
abkuerzung          0
bezeichnung         0
fk_zaehler          0
id1                 0
richtung_in         0
von                 0
objectid            0
korrekturfaktor     0
geometry            0
dtype: int64

Form: (171, 11)

Erste 5 Zeilen:
  abkuerzung              bezeichnung             bis   fk_zaehler  id1  \
0   VZS_BINZ         Binzmühlestrasse  20180404000000  ECO09113500    7   
1   VZS_BUCH             Bucheggplatz  20190313000000  ECO09113505   16   
2   VZS_KLOW         Kloster-Fahr-Weg  20200203000000  Y0412032046   53   
3   VZS_HOFW         Hofwiesenstrasse  20190313000000 

### 📝 Datenprüfung: Eco-Standorte Zürich (`taz.view_eco_standorte.json`)

Der Datensatz umfasst **171 Standorte**, die mit dem Label „Eco“ geführt werden – vermutlich Messpunkte, Anlagen oder zählbare Infrastrukturen im Zusammenhang mit Energie, Umwelt oder Verkehr. Jeder Eintrag ist mit einer Punkt-Geometrie versehen und umfasst 11 Attribute.

**Datenstruktur:**
- Die Daten enthalten verschiedene Bezeichnungen (`bezeichnung`), Kürzel (`abkuerzung`) und Zählerkennungen (`id`).
- Die Spalten `von` und `bis` repräsentieren vermutlich Zeitstempel im Format `yyyymmddHHMMSS`, z.B. zur Erfassung von Messzeiträumen.
- Die Spalte `korrekturfaktor` liegt als `float` vor und scheint auf eine Gewichtung oder Umrechnungsgröße hinzuweisen.

**Nullwerte:**
- Die Spalte `bis` ist bei **26 von 171 Einträgen leer**, möglicherweise weil die Messung noch aktiv ist.
- `richtung_out` fehlt in 4 Fällen, was auf ungerichtete Messstationen oder irrelevante Angaben hindeuten könnte.
- Alle anderen Felder sind vollständig gefüllt.

**Doppelte Einträge:**
- **Keine doppelten Zeilen** vorhanden.
- **90 doppelte Geometrien** → Das bedeutet, dass sich mehrere Einträge **denselben Standort teilen**, z.B. für unterschiedliche Zeiträume, Richtungen oder Zähler.

**Eindeutige Werte:**
- Die Spalten `id1` und `objectid` sind eindeutig → gut als Schlüssel verwendbar.
- Es gibt **110 eindeutige „von“-Werte** und **90 unterschiedliche „bis“-Werte**, was auf verschiedene Start- und Endzeitpunkte hindeutet.
- `korrekturfaktor` zeigt 42 verschiedene Werte und könnte für die spätere **gewichtete Analyse** genutzt werden.

**Fazit:**
Der Datensatz bietet eine strukturierte Übersicht über sogenannte **Eco-Standorte**, die sich z.B. als **Verkehrszählstellen, Umweltmesspunkte oder ökologisch relevante Infrastruktur** interpretieren lassen. Die Vielzahl doppelter Geometrien legt nahe, dass an einzelnen Standorten mehrere Zeitabschnitte oder Richtungsangaben erfasst wurden. Für räumliche Analysen ist der Datensatz direkt nutzbar, insbesondere zur **Verknüpfung mit Verkehrs-, Umwelt- oder Infrastrukturthemen**.


In [43]:
# Nur relevante Spalten auswählen
eco_reduced = eco[[
    "id1",
    "bezeichnung",
    "bis",
    "richtung_in",
    "richtung_out",
    "von",
    "geometry"
]].copy()

# Spalte 'id1' zu 'id' umbenennen
eco_reduced.rename(columns={"id1": "id"}, inplace=True)

# Überblick über das Ergebnis
print("Neue Form:", eco_reduced.shape)
print("\nErste 5 Zeilen:")
print(eco_reduced.head())


Neue Form: (171, 7)

Erste 5 Zeilen:
   id              bezeichnung             bis       richtung_in  \
0   7         Binzmühlestrasse  20180404000000          Oerlikon   
1  16             Bucheggplatz  20190313000000  Hofwiesenstrasse   
2  53         Kloster-Fahr-Weg  20200203000000       Stadtgrenze   
3  13         Hofwiesenstrasse  20190313000000      Bucheggplatz   
4  61  Limmatquai --> Bellevue  20190603000000          Bellevue   

      richtung_out             von                  geometry  
0  Glaubtenstrasse  20101130000000  POINT (8.52329 47.41364)  
1            Höngg  20140626000000  POINT (8.53345 47.39891)  
2       Innenstadt  20100625000000  POINT (8.48557 47.40214)  
3              ---  20100923000000  POINT (8.53397 47.40076)  
4              ---  20140625000000  POINT (8.54343 47.36912)  


In [44]:
# 'von' und 'bis' in echte datetime-Objekte umwandeln
eco_reduced["von"] = pd.to_datetime(eco_reduced["von"], format="%Y%m%d%H%M%S", errors="coerce")
eco_reduced["bis"] = pd.to_datetime(eco_reduced["bis"], format="%Y%m%d%H%M%S", errors="coerce")

# Kontrolle
print("Datentypen nach Umwandlung:")
print(eco_reduced.dtypes[["von", "bis"]])

print("\nErste Zeilen mit Datumsformat:")
print(eco_reduced[["id", "von", "bis"]].head())


Datentypen nach Umwandlung:
von    datetime64[ns]
bis    datetime64[ns]
dtype: object

Erste Zeilen mit Datumsformat:
   id        von        bis
0   7 2010-11-30 2018-04-04
1  16 2014-06-26 2019-03-13
2  53 2010-06-25 2020-02-03
3  13 2010-09-23 2019-03-13
4  61 2014-06-25 2019-06-03


In [45]:
# Spalten umbenennen
eco_reduced.rename(columns={
    "bezeichnung": "location_name",
    "richtung_in": "direction_in",
    "richtung_out": "direction_out",
    "von": "start_date",
    "bis": "end_date"
}, inplace=True)

# Neue Spaltenreihenfolge festlegen
eco_reduced = eco_reduced[[
    "id",
    "location_name",
    "direction_in",
    "direction_out",
    "start_date",
    "end_date",
    "geometry"
]]

# Überblick
print("Neue Form nach Umbenennung und Sortierung:", eco_reduced.shape)
print("\nErste 5 Zeilen:")
print(eco_reduced.head())


Neue Form nach Umbenennung und Sortierung: (171, 7)

Erste 5 Zeilen:
   id            location_name      direction_in    direction_out start_date  \
0   7         Binzmühlestrasse          Oerlikon  Glaubtenstrasse 2010-11-30   
1  16             Bucheggplatz  Hofwiesenstrasse            Höngg 2014-06-26   
2  53         Kloster-Fahr-Weg       Stadtgrenze       Innenstadt 2010-06-25   
3  13         Hofwiesenstrasse      Bucheggplatz              --- 2010-09-23   
4  61  Limmatquai --> Bellevue          Bellevue              --- 2014-06-25   

    end_date                  geometry  
0 2018-04-04  POINT (8.52329 47.41364)  
1 2019-03-13  POINT (8.53345 47.39891)  
2 2020-02-03  POINT (8.48557 47.40214)  
3 2019-03-13  POINT (8.53397 47.40076)  
4 2019-06-03  POINT (8.54343 47.36912)  


In [48]:
# Funktion zur Qualitätsprüfung eines DataFrames
def check_data_quality(df, name="DataFrame"):
    print(f"📋 Qualitätsprüfung für {name}\n")

    # Datentypen
    print("1. Datentypen:")
    print(df.dtypes)

    # Nullwerte
    print("\n2. Nullwerte pro Spalte:")
    print(df.isnull().sum().sort_values(ascending=False))

    # Form
    print("\n3. Form (Zeilen, Spalten):", df.shape)

    # Eindeutige Werte
    print("\n4. Eindeutige Werte pro Spalte:")
    print(df.nunique().sort_values(ascending=False))

    # Doppelte Zeilen
    print("\n5. Anzahl doppelter Zeilen:", df.duplicated().sum())

    # Vorschau
    print("\n6. Erste 5 Zeilen:")
    print(df.head())

    print("\n" + "-"*60 + "\n")

# Qualitätsprüfung für die Messstandorte
check_data_quality(eco_reduced, "eco_reduced (bereinigte Messstandorte)")


📋 Qualitätsprüfung für eco_reduced (bereinigte Messstandorte)

1. Datentypen:
id                        int32
location_name            object
direction_in             object
direction_out            object
start_date       datetime64[ns]
end_date         datetime64[ns]
geometry               geometry
dtype: object

2. Nullwerte pro Spalte:
end_date         26
direction_out     4
id                0
location_name     0
direction_in      0
start_date        0
geometry          0
dtype: int64

3. Form (Zeilen, Spalten): (171, 7)

4. Eindeutige Werte pro Spalte:
id               171
start_date       110
end_date          90
geometry          81
location_name     50
direction_out     37
direction_in      36
dtype: int64

5. Anzahl doppelter Zeilen: 0

6. Erste 5 Zeilen:
   id            location_name      direction_in    direction_out start_date  \
0   7         Binzmühlestrasse          Oerlikon  Glaubtenstrasse 2010-11-30   
1  16             Bucheggplatz  Hofwiesenstrasse            Höng

## 🧠 Interpretation der Qualitätsprüfung: `eco_reduced` (bereinigte Messstandorte)

Nach der gezielten Bereinigung des Eco-Standorte-Datensatzes (`eco_reduced`) zeigt die Qualitätsprüfung folgendes Ergebnis:

### 🔍 Datenstruktur:
- **Datentypen** sind korrekt:
  - Standort-ID (`id`) als `int32`,
  - Standortname und Richtungen (`location_name`, `direction_in`, `direction_out`) als `object`,
  - Start- und Enddatum (`start_date`, `end_date`) als echte `datetime64[ns]`,
  - Standort-Geometrie (`geometry`) als Punkt-Objekt.
- **Keine Nullwerte** in ID, Standortname, Startdatum oder Geometrie.
- **Nullwerte**:
  - 26 Standorte haben kein `end_date` → Diese Messungen sind vermutlich **aktuell noch aktiv**.
  - 4 Einträge ohne `direction_out` → Einseitige oder ungerichtete Messungen möglich.
- **Keine doppelten Zeilen** im Datensatz.

### 🛠️ Vorgehen und Sinnhaftigkeit:
- Durch die gezielte Reduktion auf nur relevante Spalten (`id`, Standortname, Richtungen, Zeit, Geometrie) wurde der Datensatz **kompakt und übersichtlich** aufgebaut.
- Die Umwandlung von `start_date` und `end_date` in echte Datumsformate ermöglicht später **zeitbasierte Filter und Analysen** (z.B. aktive Standorte zu einem bestimmten Zeitpunkt).
- Die saubere englische Umbenennung der Spalten erhöht die **Lesbarkeit und Wiederverwendbarkeit** des Datensatzes, insbesondere für künftige Analysen und Visualisierungen.

### ✅ Fazit:
Der Datensatz `eco_reduced` ist vollständig, sauber strukturiert und ideal vorbereitet, um später mit den effektiven Verkehrszähldaten (`fk_standort`) zusammengeführt zu werden.


In [46]:
# Speichern des bereinigten Eco-Standorte-Datensatzes als CSV
eco_reduced.to_csv(
    "../../data/Cleaned/messstandorte.csv",
    index=False,
    encoding="utf-8"
)

print("✅ Der bereinigte Eco-Standorte-Datensatz wurde erfolgreich als CSV gespeichert!")


✅ Der bereinigte Eco-Standorte-Datensatz wurde erfolgreich als CSV gespeichert!


In [49]:
# Liste der übrig gebliebenen Standorte
standorte_fussverkehr = [3015, 4246, 3279]

# Prüfen, ob diese IDs im eco_reduced vorhanden sind
vorhanden = eco_reduced[eco_reduced["id"].isin(standorte_fussverkehr)]

print("Übereinstimmende Standorte im eco_reduced:")
print(vorhanden[["id", "location_name", "direction_in", "direction_out"]])

Übereinstimmende Standorte im eco_reduced:
       id   location_name   direction_in direction_out
117  3015  Cassiopeiasteg    Stadtgrenze      Bellevue
141  3279      Limmatquai        Rathaus       Seeufer
152  4246         Dammweg  Limmatstrasse      Sihlquai


In [ ]:
#